# 03 함수적 자료구조

* 스칼라로 배우는 함수형 프로그래밍 - 3장
* 김무성

# 차례

* 3.1 함수적 자료구조의 정의
* 3.2 패턴 부합
* 3.3 함수적 자료구조의 자료 공유
    - 3.3.1 자료 공유의 효율성
    - 3.3.2 고차 함수를 위한 형식 추론 개선
* 3.4 목록에 대한 재귀와 고차 함수로의 일반화
    - 3.4.1 그 외의 목록 조작 함수들
    - 3.4.2 단순 구성요소들로 목록 함수를 조립할 때의 효율성 손실
* 3.5 트리

# 3.1 함수적 자료구조의 정의

* 함수적 자료구조란 오직 순수 함수만으로 조작되는 자료구조다.
* 함수적 자료구조는 불변이(immutable)이다.
* 이러면 여분의 복사가 많이 일어나지 않을까? (자료구조끼리 더하면 합쳐진 새로운 자료구조를 반환하는데) -> "그렇지 않다" - 왜? 자료공유 방식을 사용하므로.

## 단일 연결 목록 예제(Single Linked List) 

#### 스칼라 문법 참고

* trait : 추상 인터페이스
* case 
* object  
* 형식매개변수 : [A]
* 공변(covariant) 매개변수임을 뜻하는 가변 지정자(variance annotation) : +
* 동반 객체(companion object) : 동반 객체는 자료 형식가 같은 이름의 object로, 자료 형식의 값들을 생성하거나 조작하는 여러 편의용 함수들을 담는 목적으로 쓰인다. 

<img src="figures/sll.png" width=600 />

In [1]:
sealed trait List[+A] 
case object Nil extends List[Nothing]
case class Cons[+A](head: A, tail: List[A]) extends List[A]

object List {
    def sum(ints: List[Int]) : Int = ints match {
        case Nil => 0
        case Cons(x,xs) => x + sum(xs)
    }
    
    def product(ds: List[Double]) : Double = ds match {
        case Nil => 1.0
        case Cons(0.0, _) => 0.0
        case Cons(x, xs) => x * product(xs)
    }
    
    def apply[A](as: A*): List[A] = {
        if (as.isEmpty) Nil
        else Cons(as.head, apply(as.tail: _*))
    }

}

<img src="figures/list3.1.png" width=600 />

### 자료 생성자의 사용 예시

In [2]:
val ex1: List[Double] = Nil

Nil

In [3]:
val ex2: List[Int] = Cons(1, Nil)

Cons(1,Nil)

In [4]:
val ex3: List[String] = Cons("a", Cons("b", Nil))

Cons(a,Cons(b,Nil))

# 3.2 패턴 부합

#### 스칼라 문법 참고

* 동반 객체(companion object) : 동반 객체는 자료 형식가 같은 이름의 object로, 자료 형식의 값들을 생성하거나 조작하는 여러 편의용 함수들을 담는 목적으로 쓰인다. 
* 패턴 부합 구문 :
    - 표현식(대상target) match { case 패턴 => 결과 }
    - 예) 
        - List(1,2,3) match { case Cons(h,_) => h} 
        - 결과는 1 

In [5]:
def sum(ints: List[Int]) : Int = ints match {
    case Nil => 0
    case Cons(x,xs) => x + sum(xs)
}
    
def product(ds: List[Double]) : Double = ds match {
    case Nil => 1.0
    case Cons(0.0, _) => 0.0
    case Cons(x, xs) => x * product(xs)
}

<img src="figures/ml.png" width=600 />

In [6]:
List(1,2,3) match { case _ => 42}

42

In [7]:
List(1,2,3) match { case Cons(h,_) => h }

1

In [8]:
List(1,2,3) match { case Cons(_,t) => t }

Cons(2,Cons(3,Nil))

In [9]:
// 실행시점 MatchError 오류
// MatchError는 부합 표현식의 경우 문 중 대상과 부합하는 것이 하나도 없음을 뜻함
// 패텬이 표현식과 부합하는지 판정하는 규칙 
//  - 변수는 모든 것에 부합
//  - 자료 생성자는 해당 형태의 값에만 부합
//  - 패턴으로소 Nil은 오로지 Nil 값에만 부합, 패턴으로서 Cons(h,t)나 Cons(x,xs)는 오직 Cons 값들에만 부합
List(1,2,3) match { case Nil => 42 }

: 

### 연습문제 3.1

다음 패턴 부합 표현식의 결과는 무엇인가?

In [10]:
val x = List(1,2,3,4,5) match {
    case Cons(x, Cons(2, Cons(4,_))) => x
    case Nil => 42
    case Cons(x, Cons(y, Cons(3, Cons(4,_)))) => x + y
    case Cons(h,t) => h + sum(t)
    case _ => 101
}

3

# 3.3 함수적 자료구조의 자료 공유

* 자료가 불변이라면, 목록에 요소를 추가하거나 제거하는 함수는 어떻게 작성해야 할까?
* 기존 목록(xs)의 앞에 1이라는 요소를 추가하려면 Cons(1,xs)라는 새 목록을 만들면 된다. 
    - 목록은 불변이므로, xs를 복사할 필요가 없다. 재사용하면 된다.
* 이렇게 기존 요소가 재사용되는 것을 자료 공유(data sharing)이라고 한다.
* 기존 목록 mylist = Cons(x, xs)의 앞(첫) 요소를 제거하려면 그냥 xs을 돌려주면 된다.
    - 실질적인 제거는 일어나지 않는다.
* 원래의 목록은 여전히 사용가능한 상태이다. - 이를 두고, 함수적 자료구조는 영속적(persistent)이라고 말한다.

<img src="figures/share.png" width=600 />

### 연습문제 3.2

* List의 첫 요소를 제거하는 함수 tail을 구현하라. 이 함수가 상수 시간으로 실행됨을 주의할 것. Nil인 List도 지원하도록 독자의 구현을 수정하는 여러 가지 방법들도 고려해 보라. 이에 대해서는 다음 장에서 좀 더 살펴볼 것이다.

In [11]:
def tail[A](l: List[A]): List[A] = 
  l match {
    case Nil => sys.error("tail of empty list")
    case Cons(_,t) => t
  }

In [12]:
val x = List(1,2,3,4,5)
tail(x)

Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Nil)))))

### 연습문제 3.3

* 같은 맥락에서, List의 첫 요소를 다른 값으로 대체하는 함수 setHead를 구현하라.

In [ ]:
// wirte your code

## 3.3.1 자료 공유의 효율성

### 연습문제 3.4

* tail을 일반화해서, 목록에서 처음 n개의 요소를 제거하는 함수 drop을 구현하라. 이 함수의 실행시간은 제거되는 원소의 개수에만 비례함을 주의할 것. List 전체의 복사본을 만들 필요는 없다.

In [13]:
def drop[A](l: List[A], n: Int): List[A] = 
  if (n <= 0) l 
  else l match {
    case Nil => Nil
    case Cons(_,t) => drop(t, n-1) 
  }

In [14]:
drop(List(1,2,3,4,5), 2)

Cons(3,Cons(4,Cons(5,Nil)))

### 연습문제 3.5

* 주어진 술어(predicate)와 부합하는 List의 앞 요소들(prefix)을 제거하는 함수 dropWhile을 구현하라

In [15]:
def dropWhile[A](l: List[A], f: A => Boolean): List[A] = 
  l match {
    case Cons(h,t) if f(h) => dropWhile(t, f) 
    case _ => l
  }

In [16]:
dropWhile(List(1,2,3,4,5), (x:Int) => x <4)

Cons(4,Cons(5,Nil))

### 자료 공유의 놀라운 예

다음 함수는 한 목록의 요소를 다른 목록의 끝에 추가한다.

In [17]:
// 오직 첫 목록이 다 소진될 때까지만 값들을 복사한다
// 이 함수의 실행 시간과 메모리 사용량은 오직 a1의 길이에만 의존한다.
// 목록의 나머지는 그냥 a2를 가리킬 뿐이다.
// 불편이 연결 목록이 배열보다 훨씬 효율적이다.
def append[A](a1: List[A], a2: List[A]): List[A] = 
    a1 match {
        case Nil => a2
        case Cons(h, t) => Cons(h, append(t, a2))
    }

### 연습문제 3.6

* 그러나 모든 것이 효율적이지는 않다. 한 List의 마지막 요소를 제외한 모든 요소로 이루어진 List를 돌려주는 함수 init을 구현하라.
* 예를 들어 List(1,2,3,4)에 대해 init은 List(1,2,3)을 돌려주어야 한다.
* 이 함수를 tail처럼 상수 시간으로 구현할 수 없는 이유는 무엇인가?

In [ ]:
def init[A](l: List[A]): List[A]

In [ ]:
// wirte your Answer

## 3.3.2 고차 함수를 위한 형식 추론 개선

In [ ]:
// 인수 f에 익명 함수를 호출하기 위해서는 그 익명 함수의 인수의 형식을 명시해야 한다.
def dropWhile[A](l: List[A], f: A => Boolean): List[A]

In [18]:
val xs: List[Int] = List(1,2,3,4,5)

Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Nil)))))

In [19]:
// 익명 함수의 인수의 형식을 명시하고 있다. (x: Int)
val ex1 = dropWhile(xs, (x: Int) => x < 4)

Cons(4,Cons(5,Nil))

In [20]:
// 인수들을 두 그룹으로 묶으면 스칼라가 인수를 추론할 수 있다.
def dropWhile[A](as: List[A])(f: A => Boolean): List[A] = 
  as match {
    case Cons(h,t) if f(h) => dropWhile(t)(f) 
    case _ => as
  }

In [21]:
// 즉, dropWhies(xs)는 하나의 함수를 돌려주며, 그 함수를 인수 f로 호출한다. (즉 커링되었다)
val ex1 = dropWhile(xs)(x => x < 4)

Cons(4,Cons(5,Nil))

# 3.4 목록에 대한 재귀와 고차 함수로의 일반화

다음 두 함수는 하나의 함수로 일반화될 수 있다.

In [ ]:
def sum(ints: List[Int]) : Int = ints match {
    case Nil => 0
    case Cons(x,xs) => x + sum(xs)
}
    
def product(ds: List[Double]) : Double = ds match {
    case Nil => 1.0
    case Cons(0.0, _) => 0.0
    case Cons(x, xs) => x * product(xs)
}

### 고차 함수로 일반화

In [22]:
def foldRight[A,B](as: List[A], z: B)(f: (A, B) => B) : B =
    as match {
        case Nil => z
        case Cons(x, xs) => f(x, foldRight(xs, z)(f))
    }

In [23]:
def sum2(ns: List[Int]) = foldRight(ns, 0)((x,y) => x + y)

def product2(ns: List[Double]) = foldRight(ns, 1.0)(_ * _)

<img src="figures/list3.2.png" width=600 />

### 익명 함수를 위한 밑줄 표기법

<img src="figures/sh.png" width=600 />

### 평가 과정 추적(trace)

<img src="figures/trace.png" width=600 />

### 연습문제 3.7

* foldRight로 구현된 product(목록 3.2의 product2)가 0.0을 만났을 때 즉시 재귀를 멈추고 0.0을 돌려줄까? 왜 그럴까? 아니라면 왜 아닐까? foldRight를 긴 목록으로 호출했을 때 어떤 평가 단축이 일어나는지 고찰하라. 

In [ ]:
// wirte your answer

### 연습문제 3.8

* foldRight(List(1,2,3), Nil:List[Int]))(Cons(_,_))처럼 Nil과 Cons 자체를 foldRight에 전달하면 어떤 일이 발생할까? 이로부터, foldRight와 List의 자료 생성자들 사이의 관계에 관해 어떤 점을 알 수 있는가?

In [ ]:
// wirte your answer

### 연습문제 3.9

* foldRight를 이용해서 목록의 길이를 계산하라.

In [ ]:
def length[A](as: List[A]): Int

In [ ]:
// wirte your code

### 연습문제 3.10

* 이번 절의 foldRight 구현은 꼬리 재귀가 아니므로 긴 목록에 대해서는 StackOverflowError 오류가 발생한다(이를 "스택에 안전[stack-safe]하지 않다"라고 말한다). 실제로 그런지 실험해 보고, 꼬리 재귀적인 또 다른 일반적 목록 재귀 함수 foldLeft를 이전 장에서 논의한 기법들을 이용해서 작성하라.

In [ ]:
def foldLeft[A,B](as : List[A], z: B)(f: (B, A) => B): B

In [25]:
@annotation.tailrec
def foldLeft[A,B](l: List[A], z: B)(f: (B, A) => B): B = l match { 
  case Nil => z
  case Cons(h,t) => foldLeft(t, f(z,h))(f)
}

### 연습문제 3.11

* sum, product와 목록의 길이를 계산하는 함수를 foldLeft를 이용해서 작성하라.

In [ ]:
// wirte your code - sum

In [ ]:
// wirte your code - product

In [ ]:
//  wirte your code - length

## 3.4.1 그 외의 목록 조작 함수들

### 연습문제 3.16

* 정수 목록의 각 요소에 1을 더해서 목록을 변환하는 함수를 작성하라.(주의: 새 List를 돌려주는 순수 함수이어야 한다.)

### 연습문제 3.17

* List[Double]의 각 값을 String으로 변환하는 함수를 작성하라. d: Double을 String으로 변환할 때에는 d.toString이라는 표현식을 사용하면 된다.

### 연습문제 3.18

* 목록의 구조를 유지하면서 목록의 각 요소를 수정하는 작업을 일반화한 함수 map을 작성하라.

In [ ]:
def map[A, B](as: List[A])(f: A => B): List[B]

In [ ]:
// wirte your code 

### 연습문제 3.19

* 목록에서 주어진 술어를 만족하지 않는 요소를 제거하는 함수 filter를 작성하라. 그리고 그 함수를 이용해서 List[Int]에서 모든 홀수를 제거하라.

In [ ]:
def filter[A](as : List[A])(f: A => Boolean) : List[A]

In [ ]:
// wirte your code 

### 연습문제 3.20

* map과 비슷하되 하나의 요소가 아니라 목록을 최종 결과 목록에 삽입하는 함수 flatMap을 작성하라. 
* 예를 들어, flatMap(List(1,2,3))(i=>List(i,i))의 결과는 List(1,1,2,2,3,3,)이 된다.

In [ ]:
def flatMat[A, B](as: List[A])(f: A => List[B]) : List[B]

In [ ]:
// wirte your code 

### 연습문제 3.21

* flatMap을 이용해서 filter를 구현하라.

In [ ]:
// wirte your code 

## 표준 라이브러리 목록들

* List - 표준 라이브러리에 있다. 
* ::  -  Cons와 같다. 
    - 예) 1::2::Nil 
    - 이것은 Cons(1, Cons(2, Nil)) 과 같다.
* List[A]의 메서드로 정의되어 있는 것들 (일부만)
    - take
    - takeWhile
    - forall
    - exists
    - scanLeft
    - scanRight

## 3.4.2 단순 구성요소들로 목록 함수를 조립할 때의 효율성 손실

항상 효율적인 것은 아니다.

### 연습문제 3.24

* 어려움: 효율성 손실의 한 예로, List가 또 다른 List를 부분 순차열로 담고 있는지 점검하는 hasSubsequence 함수를 구현하라.
* 예를 들어 List(1,2)나 List(2,3), List(4)는 List(1,2,3,4)의 부분 순차열이다.
* 참고 : 스칼라에서 임의의 두 값 x와 y의 상등(equality)을 비교하는 표현식은 x == y 이다.

In [ ]:
def hasSubsequence[A](sup: List[A], sub: List[A]) : Boolean

In [ ]:
// wirte your code 

# 3.5 트리

## ADT

* List나 Tree는 대수적 자료 형식(algebraic data type, ADT)의 일종이다.
* (주의: 이것은 추상 자료 형식(abstract data type)과는 다르다)
* ADT는 하나 이상의 자료 생성자들로 이루어진 자료 형식을 말한다.
* 각 자료 생성자들은 각각 0개 이상의 인수를 받을 수 있다.
* 이러한 자료 형식을 해당 자료 생성자들의 합(sum) 또는 합집합(union)이라고 부른다.
* 각각의 자료 생성자는 해당 인수들의 곱(product)이라고 부른다.

## 이진 트리 자료구조

In [28]:
sealed trait Tree[+A]
case class Leaf[A](value: A) extends Tree[A]
case class Branch[A](left: Tree[A], right: Tree[A]) extends Tree[A]

<img src="figures/tree.png" width=600 />

### 연습문제 3.25

* 트리의 노드, 즉 잎(leaf)과 가지(branch)의 개수를 세는 함수 size를 작성하라.

In [29]:
def size[A](t: Tree[A]): Int = t match {
  case Leaf(_) => 1
  case Branch(l,r) => 1 + size(l) + size(r)
}

### 연습문제 3.26

* Tree[Int]에서 가장 큰 요소를 돌려주는 함수 maximum을 작성하라. (스칼라에서 두 정수 x와 y의 최댓값은 x.max(y)나 x max y 로 계산할 수 있다)

In [30]:
def maximum(t: Tree[Int]): Int = t match {
  case Leaf(n) => n
  case Branch(l,r) => maximum(l) max maximum(r)
}

# 참고자료

* [1] 스칼라로 배우는 함수형 프로그래밍 - http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791185890180
* [2] 책 예제 코드 github - https://github.com/fpinscala/fpinscala